In [1]:
import numpy as np
import pandas as pd
import glob
import ipyparallel as p
import os
import ntpath
import ProgressBar as pb
from pymail import alert
import time

In [2]:
exname = 'smex7_sen'
method = '7'
site = 'Nr1'

In [3]:
files = glob.glob('/Volumes/data/RHESSys_out/Nwt/%s/*_basin.daily'%(exname))

In [4]:
files[-10:]

['/Volumes/data/RHESSys_out/Nwt/smex7_sen/smex7_sen_99994_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex7_sen/smex7_sen_99995_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex7_sen/smex7_sen_99996_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex7_sen/smex7_sen_99997_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex7_sen/smex7_sen_99998_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex7_sen/smex7_sen_99999_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex7_sen/smex7_sen_9999_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex7_sen/smex7_sen_999_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex7_sen/smex7_sen_99_basin.daily',
 '/Volumes/data/RHESSys_out/Nwt/smex7_sen/smex7_sen_9_basin.daily']

In [5]:
params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

In [6]:
params.head()

,MAP,PeakSWE,SMR,TpeakSWE,combo,idx,ksat,m,pa,po,ref,scenidx
0,6.165935,0.41366,0.007447,180.0,d1:s1,0,10.0245,4.9592,0.00356,0.032637,2010,0
1,6.165935,0.41366,0.007447,180.0,d1:s2,0,10.0245,4.9592,0.00534,0.032637,2010,1
2,6.165935,0.41366,0.007447,180.0,d1:s3,0,10.0245,4.9592,0.00712,0.032637,2010,2
3,6.165935,0.41366,0.007447,180.0,d1:s4,0,10.0245,4.9592,0.00890,0.032637,2010,3
4,6.165935,0.41366,0.007447,180.0,d1:s5,0,10.0245,4.9592,0.01068,0.032637,2010,4


In [7]:
files = pd.DataFrame({'basin_daily':files})

In [8]:
def getIDX(fl):
    return int(fl.split('_')[-2])

In [9]:
files['scenidx'] = files.basin_daily.map(getIDX)

In [10]:
params = pd.merge(params,files, on= 'scenidx')

In [11]:
from rhessys import snowmelt_experiment as smex

In [12]:
params['Tnosnow'] = params.TpeakSWE+np.ceil(params.PeakSWE/params.SMR)

In [13]:
params['date_peakSWE'] = params.apply(smex.DOPEAKSWE2cal,axis=1)
params['date_ONS'] =params.apply(smex.DONS2cal,axis=1)

In [14]:
c = p.Client()
view = c.load_balanced_view()

In [15]:
c.ids

[0, 1, 2, 3, 4, 5, 6]

In [16]:
len(params)

250000

In [17]:
dr = pd.date_range('2009-10-1','2010-9-30',freq='D')

In [18]:
%px dr = pd.date_range('2009-10-1','2010-9-30',freq='D')

In [19]:
def processSM(bd,idx,strt,nd):

    # read in the basin daily data
    data = pd.read_csv(bd,delimiter=' ')

    data.index = pd.DatetimeIndex(dr)

    data['ET'] = data.evap + data.trans
    trans = data.trans.sum()
    evap = data.evap.sum()
    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    # create full output object
    full = (idx,trans,evap,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    data = data.loc[strt:nd] # crop the data to the melt season
    
    T = data.trans.sum() # compute totoal transpiration
    E = data.evap.sum() # compute total evaporation
    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    melt = (idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    return full,melt

In [20]:
idx = 3
print params.basin_daily[idx]
processSM(params.basin_daily[idx],params.scenidx[idx],params.date_peakSWE[idx],params.date_ONS[idx])

/Volumes/data/RHESSys_out/Nwt/smex7_sen/smex7_sen_3_basin.daily


((3,
  123.284062,
  197.28817400000003,
  320.572236,
  244.34013299999998,
  244.34013299999998,
  0.0,
  135.02782100000002,
  0.0,
  546.921614,
  1510.538855712327,
  0.0061935479452054805,
  1183.6840702739769,
  0.0,
  5478.913440000021,
  7240.0,
  175.79934699999998,
  21043.223084999998,
  0.6694250219178083,
  0.8782800986301371),
 (3,
  8.220068,
  26.631236,
  34.851304,
  8.267421,
  8.267421,
  0.0,
  8.871521999999999,
  0.0,
  1.179243,
  1509.9593046666669,
  0.0,
  1183.002780000001,
  0.0,
  5478.913439999998,
  7240.0,
  23.169269,
  719.1528310000001,
  0.14504247368421055,
  0.6114263859649125))

In [21]:
#res = view.map(processSM,params.basin_daily[0:1000],params.scenidx[0:1000],params.date_peakSWE[0:1000],params.date_ONS[0:1000])
res = view.map(processSM,params.basin_daily,params.scenidx,params.date_peakSWE,params.date_ONS)

In [22]:
prog = pb.ProgressBar(len(params))

In [23]:
while res.ready() == False:
    prog.animate_ipython(res.progress)
    time.sleep(2)

[****************100%******************]  249925 of 250000 complete                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [24]:
res.done()

True

In [25]:
full,melt = zip(*res.result())

## Merge processing results with parameters dataset

In [26]:
idx,trans,evap,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*full)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,trans, evap, ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','trans','evap','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_full.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True)

In [27]:
idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*melt)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','T','E','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_melt.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True) # 

In [28]:
alert.send_alert('barnhatb@colorado.edu','Processing Nr1 %s has finished'%exname,'Your script has finished')